<a href="https://colab.research.google.com/github/attusuario/Clonechat-Telegram-Colab/blob/main/9724243669bc7bc2cf2c1ba326963916.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mass auto transcriber

Creates transcription and caption of a media collection, whether video or audio.

Activate the GPU by following the steps:

- In top menu: `Runtime > Change runtime type > Hardware accelerator: GPU`

**Free Usage Limits** (approximately)
- With GPU, transcript around 30 hours of media per week.\
- With CPU, 10 hours per day

**Transcription Speed**

- GPU, 10 minutes of video in 1 minute. (10x) \
- CPU, 10 minute of video in 10 minutes. (1x) 

In [5]:
from datetime import datetime
import os
import time

from google.colab import drive, files
from pathlib import Path
from subprocess import run
from tqdm.notebook import tqdm

In [12]:
!pip install git+https://github.com/openai/whisper.git
!pip install stable-ts
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install -U git+https://github.com/jianfch/stable-ts.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-iyis1fab
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-iyis1fab
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jianfch/stable-ts.git to /tmp/pip-req-build-n6y0xjag
  Running command git clone --filter=blob:none 

In [13]:
#@title Import libs
import stable_whisper
import whisper
import torch

In [14]:
# Connecting to a google account
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
colab_folder = r'/content/gdrive/MyDrive/Colab Notebooks/'
massautotranscriber_folder = Path(colab_folder) / 'mass_auto_transcribe/'
path_folder_input = Path(massautotranscriber_folder) / 'input'
path_folder_output = Path(massautotranscriber_folder) / 'output'

In [8]:
#@title Set folders and Createif not exists
for folder_path in [colab_folder, massautotranscriber_folder, path_folder_input, path_folder_output]:
  if not Path(folder_path).exists():    
    Path(folder_path).mkdir()

FileNotFoundError: ignored

Follow the steps:
- Access your [google drive](https://drive.google.com/)
- Access the folder `Colab Notebooks/mass_auto_transcriber/input`
- Upload the media files (video or audios) there

In [ ]:
#@title Download transcription model
print(torch.cuda.is_available())
model_type = 'large'
model = whisper.load_model(model_type)
model.device

In [ ]:
#@title Make transcription

# Get media list to be transcript

list_audiopath = [x for x in Path(path_folder_input).iterdir()]
print('Existing media:' , ', '.join([x.stem for x in list_audiopath]))

list_file_transcript = [x for x in Path(path_folder_output).iterdir()]
print('Already transcripted:', ', '.join([x.stem for x in list_file_transcript]))

list_newaudiopath = [x for x in list_audiopath if x.stem not in [y.stem for y in list_file_transcript]]
print('To be transcripted:', ', '.join([x.stem for x in list_newaudiopath]))

In [ ]:
for audio_path in tqdm(list_newaudiopath):
  start_time = datetime.now() 
  result = model.transcribe(str(audio_path), language= 'pt')
  time_elapsed = datetime.now() - start_time 
  filename = Path(audio_path).stem
  print(f'Time elapsed: {str(time_elapsed)[:-7]}, {filename}')
  # save transcription
  path_file_transcricao = Path(path_folder_output) / (filename + '.txt')
  with open(path_file_transcricao, "w") as f:
    f.write(result['text'])
  # save caption
  subtitle_path_output = Path(path_folder_output) / (filename + '.srt')
  stable_whisper.results_to_sentence_srt(result, str(subtitle_path_output))

In [ ]:
#@title Generate a zip package of transcriptions
os.system(f'zip -r transcript.zip "{path_folder_output}"')

If you wish, in your google drive, delete the input and output content

In [ ]:
#@title Download zip file
files.download("transcript.zip")

In [ ]:
#@title Delete online zip file
os.system(f'rm transcript.zip')